In [2]:
from os.path import join
import time

import einops
import numpy as np
from scipy import signal
import torch

from datasets.dreamer import DREAMERDataset
from datasets.deap import DEAPDataset
import matplotlib.pyplot as plt

# Data

In [3]:
dreamer = DREAMERDataset(path=join("..", "..", "datasets", "eeg_emotion_recognition", "dreamer"),
                         split_in_windows=True,
                         window_size=1, drop_last=True,
                         discretize_labels=True, normalize_eegs=True,
                         validation="k_fold", k_folds=10,
                         labels_to_use=["valence", "arousal", "dominance"],
                         batch_size=128)

In [4]:
deap = DEAPDataset(path=join("..", "..", "datasets", "eeg_emotion_recognition", "deap"),
                         split_in_windows=True,
                         window_size=1, drop_last=True,
                         discretize_labels=True, normalize_eegs=True,
                         validation="k_fold", k_folds=10,
                         labels_to_use=["valence", "arousal", "dominance"],
                         batch_size=128)

In [5]:
print(len(deap), len(dreamer))

76800 85744


In [8]:
print(dreamer.eegs_data.min(), dreamer.eegs_data.max())
print(deap.eegs_data.min(), deap.eegs_data.max())

tensor(-8.1110e-05) tensor(0.0001)
tensor(-4.2283e-05) tensor(3.5265e-05)


# Decompositions

## Wavelets

In [6]:
def wavelet_decompose(x, widths):
    assert any([isinstance(widths, t) for t in {np.ndarray, torch.Tensor, list}])
    assert all([width > 0 for width in widths])
    x_decomposed = torch.stack([torch.as_tensor(signal.cwt(x[:, i_channel], signal.ricker, widths))
                                for i_channel in range(x.shape[-1])], dim=-1)  # (w, s, e)
    return x_decomposed


s = wavelet_decompose(x=sample_eeg, widths=[1, 2, 4, 8, 16, 32])
print(s.shape)

NameError: name 'sample_eeg' is not defined

In [ ]:
for i in s:
    plt.plot(i)
    plt.show()

## Spectrogram

In [ ]:
import librosa


def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(specgram, origin="lower", aspect="auto")
    fig.colorbar(im, ax=axs)
    plt.show(block=False)


def get_mel_spectrogram(x, sampling_rate: Union[int, float]):
    transform = transforms.MelSpectrogram(sample_rate=128, f_min=3, f_max=50, n_fft=50, n_mels=8, center=True,
                                          win_length=8)
    mel_spectrogram = transform(einops.rearrange(x, "s c -> c s"))
    return mel_spectrogram

In [ ]:
from torch import nn
from torchaudio import transforms

print(sample_eeg.shape)
st = time.time()
s = get_mel_spectrogram(x=sample_eeg)
print(s.shape)
plot_spectrogram(s[0])
# plot_spectrogram(mel_specgram[0], title="MelSpectrogram - torchaudio", ylabel="mel freq")
print(time.time() - st)